In [1]:
from bs4 import BeautifulSoup
import requests

# Extract the content of the HTML page and create soup object for scrapping

In [2]:
text = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(text, "lxml")

# Create object table from HTML tag "table" with class attribute "wikitable sortable"

In [3]:
table = soup.find('table',{'class':'wikitable sortable'})


# Extract all the table rows from the HTML table and remove the headers row

In [4]:
rows = table.find_all('tr')
rows.pop(0)

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>

# Logic to create the dataframe with the specified requirements

In [18]:
import pandas as pd
l = []
i=0
    
for tr in rows:
    td = tr.find_all('td')
    
    if td[1].text != 'Not assigned':
        
        if td[2].text == 'Not assigned\n':
            td[2] = td[1]
            
        row = [tr.text for tr in td]
        row[2] = row[2].rstrip()
        
        if i == 0:
            l.append(row)
            i = i+1
            continue

            
        else:
            if l[i-1][1] == row[1]:
                l[i-1][2] =  l[i-1][2] +","+row[2]
                continue

        i = i+1
        l.append(row)
        
df = pd.DataFrame(l, columns=["Postal Code", "Borough", "Neighborhood"])
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,"Parkwoods,Victoria Village"
1,M5A,Downtown Toronto,"Harbourfront,Regent Park"
2,M6A,North York,"Lawrence Heights,Lawrence Manor"
3,M7A,Queen's Park,Queen's Park
4,M9A,Etobicoke,Islington Avenue


# Shape of the dataframe

In [6]:
df.shape

(85, 3)

# Part II

# Download csv file with coordinates and create a second dataframe

In [7]:
df2 = pd.read_csv("http://cocl.us/Geospatial_data")
df2 = pd.merge(df, df2, on='Postal Code')
df2.head()
df2.shape

(85, 5)

# Part III

# Create function to do foursquare calls

In [8]:
client_id='BIL0PT3S5OOZK4TAMF5GOYIGP5X35KQQ5BR1IHH3RNO20GI1'
client_secret='FDTSTTWTC4DD21FBWF2M1CQLACRKZKUUYXU4XRDXVBY3PSE5'
version = '20180605'
limit = 100

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            client_id, 
            client_secret, 
            version, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Call for toronto venues

In [10]:
toronto_venues = getNearbyVenues(names=df2['Neighborhood'],
                                   latitudes=df2['Latitude'],
                                   longitudes=df2['Longitude'])

Parkwoods,Victoria Village
Harbourfront,Regent Park
Lawrence Heights,Lawrence Manor
Queen's Park
Islington Avenue
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street,Christie
Cedarbrae
Hillcrest Village,Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole,Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village,CFB Toronto,Downsview East
The Danforth West,Riverdale
Design E

# Analize each neighborhood

In [11]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.000000,0.000000,0.000000,0.0000,0.0,0.010000,0.000000,0.000000,0.010000
1,Agincourt,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
3,"Alderwood,Long Branch,Northwest",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
4,"Bayview Village,CFB Toronto,Downsview East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
5,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
6,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
7,"Birch Cliff,Cliffside West",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
8,"Bloordale Gardens,Eringate,Markland Wood,Old B...",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000
9,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0000,0.0,0.000000,0.000000,0.000000,0.000000


# Cluster Neighborhoods

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [14]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Hotel,Bakery,Asian Restaurant,Restaurant,Gym
1,Agincourt,Sandwich Place,Breakfast Spot,Lounge,Skating Rink,Chinese Restaurant,Women's Store,Donut Shop,Diner,Discount Store,Dog Run
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Playground,Park,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Deli / Bodega,Donut Shop
3,"Alderwood,Long Branch,Northwest",Pizza Place,Coffee Shop,Pub,Skating Rink,Pharmacy,Gym,Sandwich Place,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
4,"Bayview Village,CFB Toronto,Downsview East",Café,Chinese Restaurant,Bank,Japanese Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Dessert Shop
5,"Bedford Park,Lawrence Manor East",Italian Restaurant,Thai Restaurant,Fast Food Restaurant,Coffee Shop,Pizza Place,Pharmacy,Butcher,Café,Restaurant,Pub
6,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Steakhouse,Farmers Market,Bakery,Cheese Shop,Pub,Café,Italian Restaurant
7,"Birch Cliff,Cliffside West",College Stadium,Café,General Entertainment,Skating Rink,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
8,"Bloordale Gardens,Eringate,Markland Wood,Old B...",Pizza Place,Shopping Plaza,Beer Store,Liquor Store,Café,Pharmacy,Comic Shop,Concert Hall,Event Space,College Stadium
9,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Breakfast Spot,Café,Pet Store,Climbing Gym,Burrito Place,Stadium,Bar,Italian Restaurant,Furniture / Home Store


In [15]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

In [25]:
toronto_merged = df2.iloc[:82,:]

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,"Parkwoods,Victoria Village",43.753259,-79.329656,1,Park,Food & Drink Shop,Fast Food Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Falafel Restaurant,Dessert Shop,Dumpling Restaurant
1,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,1,Coffee Shop,Café,Bakery,Park,Pub,Breakfast Spot,Mexican Restaurant,Theater,Italian Restaurant,Bank
2,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763,4,Clothing Store,Furniture / Home Store,Women's Store,Accessories Store,Boutique,Event Space,Athletics & Sports,Vietnamese Restaurant,Coffee Shop,Shoe Store
3,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Gym,Diner,Yoga Studio,Bar,Italian Restaurant,Japanese Restaurant,Café,Spa
4,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  49.61 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  29.83 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  29.81 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  40.10 MB/s


In [29]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[43.70011, -79.4163], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters